In [1]:
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, ELU
from keras import backend as K
from keras.layers.core import Flatten, Dense, Dropout, Lambda
import pickle
from scipy import misc
from scipy.misc import imresize
from PIL import Image
import glob, os
import cv2
import csv
import math
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.layers import Convolution2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
import json

Using TensorFlow backend.


In [2]:
learning_rate = 0.0001
batch_size = 128
nb_epoch = 5

In [3]:
def normalize(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [-0.5, 0.5]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    #costants
    a = -0.5
    b = 0.5
    xmin = np.min(image_data)
    xmax = np.max(image_data) 
    
    x = image_data    
    x_prime = a + ((x-xmin)*(b-a))/(xmax-xmin)
    
    return x_prime



In [4]:
images_list = []

for infile in glob.glob("IMG/center*.jpg"):
    image = misc.imread(infile)
    image = misc.imresize(image, (66,200))
    images_list.append(image)
    
X_train = np.stack(images_list)
print(X_train.shape)

(8036, 66, 200, 3)


In [5]:
with open('driving_log.csv','r') as file:
    datareader = csv.reader(file,delimiter=',')
    driving_log = []
    for row in datareader:
        driving_log.append(row)
        
driving_log.pop(0)
        
log = np.stack(driving_log)
print(log.shape)

#labels
y_train = np.zeros(log.shape[0])
for i,row in enumerate(log):
    y_train[i] = log[i][3]
    
    
print (y_train.shape)

(8036, 7)
(8036,)


In [7]:
x_train_normalized = []
for image_data in X_train:
    normalized_image = normalize(image_data)
    x_train_normalized.append(normalized_image)
    

print(X_train.shape)
print('Data normalized')

x_train_normalized = map(normalize, X_train)
print (len(list(x_train_normalized)))



(8036, 66, 200, 3)
Data normalized
8036


In [8]:
model = Sequential()
#model.add(Lambda(lambda x:x/127.5 -1., input_shape = (66,200,3)))
model.add(Convolution2D(24, 5, 5,border_mode='valid',subsample=(2,2), dim_ordering='tf', input_shape=(66,200,3), init='he_normal'))
model.add(ELU())
model.add(Convolution2D(36, 5, 5, border_mode='valid',subsample=(2,2), init='he_normal'))
model.add(ELU())
model.add(Convolution2D(48, 5, 5, border_mode='valid',subsample=(2,2),init='he_normal'))
model.add(ELU())
model.add(Convolution2D(64, 3, 3, border_mode='valid',subsample=(1,1),init='he_normal'))
model.add(ELU())
model.add(Convolution2D(64, 3, 3, border_mode='valid',subsample=(1,1),init='he_normal'))
model.add(Flatten())
model.add(ELU())
model.add(Dense(1164,init='he_normal'))
model.add(ELU())
model.add(Dense(100,init='he_normal'))
model.add(ELU())
model.add(Dense(50,init='he_normal'))
model.add(ELU())
model.add(Dense(10,init='he_normal'))
model.add(ELU())
model.add(Dense(1,name='output',init='he_normal'))

In [9]:
optimizer = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, validation_split=0.3, shuffle=True)

Train on 5625 samples, validate on 2411 samples
Epoch 1/5
5625/5625 [==============================] - 52s - loss: 0.6162 - acc: 0.4891 - val_loss: 0.0186 - val_acc: 0.5367
Epoch 2/5
5625/5625 [==============================] - 51s - loss: 0.0180 - acc: 0.5452 - val_loss: 0.0188 - val_acc: 0.5367
Epoch 3/5
5625/5625 [==============================] - 50s - loss: 0.0174 - acc: 0.5452 - val_loss: 0.0202 - val_acc: 0.5367
Epoch 4/5
5625/5625 [==============================] - 50s - loss: 0.0173 - acc: 0.5452 - val_loss: 0.0184 - val_acc: 0.5367
Epoch 5/5
5625/5625 [==============================] - 52s - loss: 0.0171 - acc: 0.5452 - val_loss: 0.0171 - val_acc: 0.5367


In [10]:
print(history.history['acc'][-1])

0.545244444338


In [11]:
json_string = model.to_json()
with open('model.json','w') as f:
    json.dump(json_string,f,ensure_ascii=False)

model.save_weights('model.h5')